In [1]:
import pandas as pd 

import numpy as np 
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

from pandas.api import types

In [2]:
data = pd.read_csv("src/did_training_productivity.csv", index_col = 0)

Estimate the simple 2x2 Difference-in-Differences (DiD) model without covariates:

$$ Y_{it} = \alpha + \beta W_i + \gamma \text{Post}_t + \theta (W_i \times \text{Post}_t) + \epsilon_{it} $$

Report results for both the extensive margin treatment and intensive margin (replace treatment indicator with hours).

In [3]:
# extensive margin_treatment
data22 = data.copy()
data22["interaction_post"] = data22["post"]*data22["treat_group"]

y = data22["productivity"]
X_extensive = data22[["treat_group","post", "interaction_post"]]

did22_extensive = sm.OLS(y, sm.add_constant(X_extensive)).fit()

In [4]:
did22_extensive.get_robustcov_results(cov_type = "HAC", maxlags = 24).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           productivity   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     1828.
Date:                Sun, 06 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:03:06   Log-Likelihood:            -2.1204e+05
No. Observations:               48000   AIC:                         4.241e+05
Df Residuals:                   47996   BIC:                         4.241e+05
Df Model:                           3                                         
Covariance Type:                  HAC                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              126.5662      0.401    315.261      0.000     125.779     127.353
treat_group          2.8069      0.757      3.706      0.000       1.323       4.291
post                13.0608      0.369     35.358      0.000      12.337      13.785
interaction_post    22.4094      0.699     32.040      0.000      21.039      23.780
==============================================================================
Omnibus:                        7.125   Durbin-Watson:                   0.679
Prob(Omnibus):                  0.028   Jarque-Bera (JB):                7.108
Skew:                          -0.025   Prob(JB):                       0.0286
Kurtosis:                       2.969   Cond. No.                         6.35
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 24 lags and without small sample correction
"""

In [5]:
data22["interaction_hours"] = data22["post"]*data22["treat_hours"]
X_intensive = data22[["treat_group","post", "interaction_hours"]]

did22_intensive = sm.OLS(y, sm.add_constant(X_intensive)).fit()

In [6]:
did22_intensive.get_robustcov_results(cov_type = "HAC", maxlags = 24).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           productivity   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                     1871.
Date:                Sun, 06 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:03:08   Log-Likelihood:            -2.1211e+05
No. Observations:               48000   AIC:                         4.242e+05
Df Residuals:                   47996   BIC:                         4.243e+05
Df Model:                           3                                         
Covariance Type:                  HAC                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               126.2771      0.399    316.236      0.000     125.494     127.060
treat_group           3.7724      0.743      5.078      0.000       2.316       5.228
post                 13.6391      0.359     37.992      0.000      12.935      14.343
interaction_hours     1.0285      0.031     32.745      0.000       0.967       1.090
==============================================================================
Omnibus:                        6.823   Durbin-Watson:                   0.682
Prob(Omnibus):                  0.033   Jarque-Bera (JB):                6.821
Skew:                          -0.026   Prob(JB):                       0.0330
Kurtosis:                       2.972   Cond. No.                         27.2
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 24 lags and without small sample correction
"""

Estimate the two-way fixed effects model:

$$ Y_{it} = \alpha_i + \delta_t + \theta (W_i \times \text{Post}_t) + \epsilon_{it} $$

Compare results with the simple DiD.

In [7]:
fe_2way_data = data22.reset_index().copy()
fe_2way_data = fe_2way_data[["worker_id", "period", "interaction_post", "productivity"]].copy()

In [8]:
fe_2way_data = pd.get_dummies(data = fe_2way_data, 
                              columns= ["worker_id"],
                              drop_first= True)

fe_2way_data = pd.get_dummies(data = fe_2way_data,
                              columns= ["period"],
                              drop_first= False)

In [9]:
fe_2way_data_X = fe_2way_data.drop(["period_12", "productivity"], axis = 1)
fe_2way_data_y = fe_2way_data["productivity"]

In [20]:
fe = sm.OLS(fe_2way_data_y, fe_2way_data_X).fit()

In [21]:
(fe.params["interaction_post"],fe.pvalues["interaction_post"])


(22.219668108466173, 0.0)

In [ ]:
(did22_extensive.params["interaction_post"], did22_extensive.pvalues["interaction_post"])

(22.409441823187223, 0.0)

In [18]:
(did22_extensive.rsquared,fe.rsquared)

(0.29118589729046374, 0.993801310438346)

Conduct randomization inference:

(a) Randomly reassign treatment status 500 times

(b) Estimate the placebo treatment effect each time

(c) Compare your actual estimate to this distribution

(d) Calculate the p-value as the proportion of placebo effects larger
in absolute value than your estimate

In [24]:
data.columns

Index(['period', 'female', 'mothers_educ', 'technical', 'immigrant',
       'base_productivity', 'treat_group', 'tenure', 'prev_performance',
       'team_size', 'manager_exp', 'post', 'treated', 'treat_hours',
       'productivity', 'log_productivity'],
      dtype='object')

In [ ]:
# Number of permutations that I will run to carry out the randomization inference

# NOTE: this code is not efficient

num_permutations = 500

df_notreatment = data.copy()
df_notreatment = df_notreatment.drop(["treat_group"], axis = 1)

n_subjects = data["treat_group"].sum()

intpost_coefficients = []
treat_coefficients = []

for i in range(num_permutations):
    # Creates a permuted version of the DataFrame by randomly shuffling the rows without replacement
    permuted_df = df_notreatment.sample(frac=1, replace=False)

    # I assign to this variable the number of units to which I will have to assign the treatment in the MC simulation 
    # n_treatment = number of subjects that received treatment
    n_treatment = n_subjects

    # I assign to the first "n_treatment" rows of the permuted dataframe the treatment
    treatment_df = permuted_df.copy().iloc[:n_treatment]
    treatment_df["treat_group"] =  1

    # I leave the remaining entries untreated
    no_treatment_df = permuted_df.copy().iloc[n_treatment:]
    no_treatment_df["treat_group"] = 0

    # I concatenate the treatment and non-treatment dataframe
    # I do this, so that then I can run my function to estimate the coefficient of the treatment dummy
    permuted_df = pd.concat([treatment_df, no_treatment_df], axis = 0)

    # I estimate the treatment coefficient
    permuted_df["interaction_post"] = permuted_df["post"]*permuted_df["treat_group"]
    X_ri = permuted_df[["treat_group","post", "interaction_post"]]
    y_ri = permuted_df["productivity"]

    reg = sm.OLS(y_ri, sm.add_constant(X_ri)).fit()
    
    intpost_coefficient = reg.params['interaction_post']
    intpost_coefficients.append(intpost_coefficient)

    treat_group_coefficient = reg.params['treat_group']
    treat_coefficients.append(treat_group_coefficient)

In [38]:
stats.percentileofscore(intpost_coefficients,did22_extensive.params["interaction_post"])

100.0

In [40]:
stats.percentileofscore(treat_coefficients,did22_extensive.params["treat_group"])

100.0

In [39]:
stats.percentileofscore(intpost_coefficients,fe.params["interaction_post"])

100.0